In [2]:
####################################################### IMPORT #########################################################
from os.path import join as opj
import os
import nipype.interfaces.utility as niu
from nipype.interfaces.fsl import (BET, ExtractROI, FAST, FLIRT,
                                   FNIRT, ApplyWarp, MCFLIRT,
                                   MotionOutliers, BinaryMaths, ImageMeants,
                                   DilateImage)
from nipype.interfaces.fsl.maths import MathsCommand
from nipype.interfaces import fsl
from nipype.interfaces.afni import (Despike, TShift, Volreg,
                                    TProject, MaskTool, Refit,
                                    Fourier, Detrend, Maskave,
                                    MaskTool, TStat, Localstat,
                                    Bandpass, Calc, BlurInMask)
from nipype.interfaces.ants import (Registration, ApplyTransforms)

from nipype.interfaces.io import DataSink, SelectFiles
from nipype.interfaces.utility import IdentityInterface, Function
from nipype import Workflow, Node
from nipype.interfaces.c3 import C3dAffineTool

# this is a quick function to tell python how to find the right file from the segmentation function
pickindex = lambda x, i: x[i]

################## DEFINES THE MATLAB SCRIPT TO BPF AND DETREND CSF ############################

# This is a useful way to call in matlab to do something to a result from nipype, in this case -
# I need to bandpass and detrend regressors so they don't re-introduce noise

from nipype.interfaces.matlab import MatlabCommand
from nipype.interfaces.base import TraitedSpec, BaseInterface, BaseInterfaceInputSpec, File
import os
from string import Template


class bpfdtcsfInputSpec(BaseInterfaceInputSpec):
    in_file = File(exists=True, mandatory=True)
    out_file = File('CSF_bpfdt.1D', usedefault=True)


class bpfdtcsfOutputSpec(TraitedSpec):
    out_file = File(exists=True)


class CSFbpfdt(BaseInterface):
    input_spec = bpfdtcsfInputSpec
    output_spec = bpfdtcsfOutputSpec

    def _run_interface(self, runtime):
        d = dict(in_file=self.inputs.in_file,
                 out_file=self.inputs.out_file)
        # this is your MATLAB code template
        script = Template("""oned = load('$in_file');
        bpf = bandpass(oned, [0.01 0.08]);
        bpfdt = detrend(bpf, 2);
        save('$out_file', 'bpfdt', '-ascii');
        exit;""").substitute(d)

        mlab = MatlabCommand(script=script, mfile=True)
        result = mlab.run()

        return result.runtime

    def _list_outputs(self):
        outputs = self._outputs().get()
        outputs['out_file'] = os.path.abspath('CSF_bpfdt.1D')
        return outputs


############### DEFINES THE SAME PROCESS AS ABOVE BUT DEDICATED FOR MOTION CORRECTION  ####################


class bpfdtInputSpec(BaseInterfaceInputSpec):
    in_file = File(exists=True, mandatory=True)
    out_file = File('MOCO_bpfdt.1D', usedefault=True)


class bpfdtOutputSpec(TraitedSpec):
    out_file = File(exists=True)


class MOCObpfdt(BaseInterface):
    input_spec = bpfdtInputSpec
    output_spec = bpfdtOutputSpec

    def _run_interface(self, runtime):
        d = dict(in_file=self.inputs.in_file,
                 out_file=self.inputs.out_file)
        # this is your MATLAB code template
        script = Template("""oned = load('$in_file');
        bpf = bandpass(oned, [0.01 0.08]);
        bpfdt = detrend(bpf, 2);
        save('$out_file', 'bpfdt', '-ascii');
        exit;""").substitute(d)

        mlab = MatlabCommand(script=script, mfile=True)
        result = mlab.run()

        return result.runtime

    def _list_outputs(self):
        outputs = self._outputs().get()
        outputs['out_file'] = os.path.abspath('MOCO_bpfdt.1D')
        return outputs


############### DEFINES FUNCTION TO CONCATENATE REGRESSORS INTO SINGLE MATRIX  ####################


class concatInputSpec(BaseInterfaceInputSpec):
    in_file_a = File(exists=True, mandatory=True)
    in_file_b = File(exists=True, mandatory=True)
    out_file = File('regmodel.1D', usedefault=True)


class concatOutputSpec(TraitedSpec):
    out_file = File(exists=True)


class ConcatModel(BaseInterface):
    input_spec = concatInputSpec
    output_spec = concatOutputSpec

    def _run_interface(self, runtime):
        a = dict(in_file_a=self.inputs.in_file_a,
                 in_file_b=self.inputs.in_file_b,
                 out_file=self.inputs.out_file)
        # this is your MATLAB code template
        conscript = Template("""moco = load('$in_file_a');
        csf = load('$in_file_b');
        regmodel = horzcat(csf, moco);
        save('$out_file', 'regmodel', '-ascii');
        exit;""").substitute(a)

        z = MatlabCommand(script=conscript, mfile=True)
        res = z.run()

        return res.runtime

    def _list_outputs(self):
        outputs = self._outputs().get()
        outputs['out_file'] = os.path.abspath('regmodel.1D')
        return outputs


########################################################################################################################
#
#  $$$$$$$\                      $$$$$$$\
#  $$  __$$\                     $$  __$$\
#  $$ |  $$ | $$$$$$\   $$$$$$\  $$ |  $$ | $$$$$$\   $$$$$$\   $$$$$$$\
#  $$$$$$$  |$$  __$$\ $$  __$$\ $$$$$$$  |$$  __$$\ $$  __$$\ $$  _____|
#  $$  ____/ $$ |  \__|$$$$$$$$ |$$  ____/ $$ |  \__|$$ /  $$ |$$ /
#  $$ |      $$ |      $$   ____|$$ |      $$ |      $$ |  $$ |$$ |
#  $$ |      $$ |      \$$$$$$$\ $$ |      $$ |      \$$$$$$  |\$$$$$$$\
#  \__|      \__|       \_______|\__|      \__|       \______/  \_______|
#
#
#
############################################ INPUT SOURCING FOR MULTIPLE PT's AND SCANS #################################

fsl.FSLCommand.set_default_output_type('NIFTI_GZ')
experiment_dir = '/Users/James/Desktop/preproc/'
output_dir = 'datasink'
working_dir = 'workingdir'

# list of subject identifiers, input your subject numbers. 
# subject_list = ['01', '02', '03', '04', '05', '06', '07', '08']
# subject_list = ['09', '10', '11', '12', '13', '14', '15', '16']
# subject_list = ['17', '18', '19', '20', '21', '22', '23', '24']
# subject_list = ['25', '26', '27', '28', '29', '30', '31', '32', '33']
# subject_list = ['34', '35', '36', '37', '38', '39', '40']
# subject_list = ['41', '42', '43', '44', '45']
# subject_list = ['46', '47', '48']
# subject_list = ['49']
# subject_list = ['50']

# list of session identifiers
# ses_list = ['1']
# ses_list = ['2']
# ses_list = ['1', '2']

preproc = Workflow(name='preproc')
preproc.base_dir = '/Users/James/Desktop/preproc/'

################################################ Input - Output Stream #################################################

# Infosource - a function free node to iterate over the list of subject names
infosource = Node(IdentityInterface(fields=['subject_id', 'ses']),
                  name="infosource")
infosource.iterables = [('subject_id', subject_list),
                        ('ses', ses_list)]

# SelectFiles - to grab the data (alternative to DataGrabber)
anat_file = opj('sub-ID{subject_id}', 'ses-{ses}', 'anat', 'sub-ID{subject_id}_ses-{ses}*.nii.gz')
func_file = opj('sub-ID{subject_id}', 'ses-{ses}', 'func', 'sub-ID{subject_id}_ses-{ses}*.nii.gz')

templates = {'anat': anat_file,
             'func': func_file}
selectfiles = Node(SelectFiles(templates,
                               base_directory='/Users/James/Psilodep/BIDS/'),
                   name="selectfiles")

# Datasink - creates output folder for important outputs
datasink = Node(DataSink(base_directory=experiment_dir,
                         container=output_dir),
                name="datasink")

preproc.connect([(infosource, selectfiles, [('subject_id', 'subject_id'),
                                            ('ses', 'ses')])])

#################################################### Functional_Processing #############################################

#
# ███████╗██████╗ ██████╗  ██████╗  ██████╗
# ██╔════╝██╔══██╗██╔══██╗██╔═══██╗██╔════╝
# █████╗  ██████╔╝██████╔╝██║   ██║██║
# ██╔══╝  ██╔═══╝ ██╔══██╗██║   ██║██║
# ██║     ██║     ██║  ██║╚██████╔╝╚██████╗
# ╚═╝     ╚═╝     ╚═╝  ╚═╝ ╚═════╝  ╚═════╝

#########################################################################################################################

fproc = Workflow(name='fproc')

# ExtractROI - skip dummy scans
trim = Node(ExtractROI(t_min=3,
                       t_size=477,
                       output_type='NIFTI_GZ'),
            name="trim")

# 3dDespike - despike
despike = Node(Despike(outputtype='NIFTI_GZ', args='-NEW'),
               name="despike")
fproc.connect([(trim, despike, [('roi_file', 'in_file')])])
preproc.connect([(selectfiles, fproc, [('func', 'trim.in_file')])])

# 3dTshift - slice time correction
slicetime = Node(TShift(outputtype='NIFTI_GZ',
                        tpattern='alt+z2'),
                 name="slicetime")
fproc.connect([(despike, slicetime, [('out_file', 'in_file')])])

# 3dVolreg - correct motion and output 1d matrix
moco = Node(Volreg(outputtype='NIFTI_GZ',
                   interp='Fourier',
                   zpad=4,
                   args='-twopass'),
            name="moco")
fproc.connect([(slicetime, moco, [('out_file', 'in_file')])])

moco_bpfdt = Node(MOCObpfdt(), name='moco_bpfdt')
fproc.connect([(moco, moco_bpfdt, [('oned_file', 'in_file')])])

############################################## Registration - FSL/ANTs #################################################
#

#  ██████╗ ██████╗ ██████╗ ███████╗ ██████╗
# ██╔════╝██╔═══██╗██╔══██╗██╔════╝██╔════╝
# ██║     ██║   ██║██████╔╝█████╗  ██║  ███╗
# ██║     ██║   ██║██╔══██╗██╔══╝  ██║   ██║
# ╚██████╗╚██████╔╝██║  ██║███████╗╚██████╔╝
#  ╚═════╝ ╚═════╝ ╚═╝  ╚═╝╚══════╝ ╚═════╝

########################################################################################################################

coreg = Workflow(name='coreg')

# BET - structural data brain extraction
bet_anat = Node(BET(output_type='NIFTI_GZ',
                    frac=0.37,
                    robust=True), name="bet_anat")

# FSL segmentation process to get WM map
seg = Node(FAST(bias_iters=6,
                img_type=1,
                output_biascorrected=True,
                output_type='NIFTI_GZ'), name="seg")
coreg.connect([(bet_anat, seg, [('out_file', 'in_files')])])

# functional to structural bits
mean = Node(MCFLIRT(mean_vol=True,
                    output_type='NIFTI_GZ'), name="mean")

func2struc = Node(FLIRT(cost='bbr',
                        dof=6,
                        output_type='NIFTI_GZ'), name='func2struc')
coreg.connect([(seg, func2struc, [('restored_image', 'reference')])])
coreg.connect([(mean, func2struc, [('mean_img', 'in_file')])])
coreg.connect([(seg, func2struc, [(('tissue_class_files', pickindex, 2), 'wm_seg')])])

f2s_c3d = Node(C3dAffineTool(itk_transform=True,
                             fsl2ras=True), name='f2s_c3d')
coreg.connect([(func2struc, f2s_c3d, [('out_matrix_file', 'transform_file')])])
coreg.connect([(mean, f2s_c3d, [('mean_img', 'source_file')])])
coreg.connect([(seg, f2s_c3d, [('restored_image', 'reference_file')])])

# Functional to structural registration via ANTs non-linear registration

reg = Node(Registration(fixed_image='/usr/local/fsl/data/standard/MNI152_T1_2mm_brain.nii.gz',
                        transforms=['Affine', 'SyN'],
                        transform_parameters=[(0.1,), (0.1, 3.0, 0.0)],
                        number_of_iterations=[[1500, 1000, 1000], [100, 70, 50, 20]],
                        dimension=3,
                        write_composite_transform=True,
                        collapse_output_transforms=True,
                        metric=['MI'] + ['CC'],
                        metric_weight=[1] * 2,
                        radius_or_number_of_bins=[32] + [4],
                        convergence_threshold=[1.e-8, 1.e-9],
                        convergence_window_size=[20] + [10],
                        smoothing_sigmas=[[2, 1, 0], [4, 2, 1, 0]],
                        sigma_units=['vox'] * 2,
                        shrink_factors=[[4, 2, 1], [6, 4, 2, 1]],
                        use_histogram_matching=[False] + [True],
                        use_estimate_learning_rate_once=[True, True],
                        output_warped_image=True),
           name='reg')

coreg.connect([(seg, reg, [('restored_image', 'moving_image')])])

merge1 = Node(niu.Merge(2), iterfield=['in2'], name='merge1')
coreg.connect([(f2s_c3d, merge1, [('itk_transform', 'in2')])])
coreg.connect([(reg, merge1, [('composite_transform', 'in1')])])

warp = Node(ApplyTransforms(reference_image='/usr/local/fsl/data/standard/MNI152_T1_2mm_brain.nii.gz',
                            input_image_type=3), name='warp')
coreg.connect([(moco, warp, [('out_file', 'input_image')])])
coreg.connect([(merge1, warp, [('out', 'transforms')])])

# This code would be for FSL ONLY non-linear registration. This does not perform as well as ANTs, so use the ANTs reg above if possible

# see https://www.ncbi.nlm.nih.gov/pubmed/19195496 for performance of ANTs SyN used here

# flirt1 = Node(FLIRT(reference = '/usr/local/fsl/data/standard/MNI152_T1_2mm_brain.nii.gz',
#                     output_type = 'NIFTI_GZ'), name = "flirt1")
# coreg.connect([(seg, flirt1, [('restored_image', 'in_file')])])
#
# flirt2mni = Node(FLIRT(reference = '/usr/local/fsl/data/standard/MNI152_T1_2mm_brain.nii.gz',
#                        output_type = 'NIFTI_GZ'), name = "flirt2mni")
# coreg.connect([(seg, flirt2mni, [('restored_image', 'in_file')])])
# coreg.connect([(flirt1, flirt2mni, [('out_matrix_file', 'in_matrix_file')])])
#
# fnirt2mni = Node(FNIRT(fieldcoeff_file = True,
#                        ref_file = '/usr/local/fsl/data/standard/MNI152_T1_2mm.nii.gz',
#                        subsampling_scheme = [8,4,2,2],
#                        output_type = 'NIFTI_GZ'), name = "fnirt2mni")
# coreg.connect([(flirt2mni, fnirt2mni, [('out_matrix_file', 'affine_file')])])
# coreg.connect([(seg, fnirt2mni, [('restored_image', 'in_file')])])
#
#
# applywarp = Node(ApplyWarp(ref_file = '/usr/local/fsl/data/standard/MNI152_T1_2mm_brain.nii.gz',
#                            output_type = 'NIFTI_GZ',
#                            args = '-v'), name = "applywarp")
# coreg.connect([(func2struc, applywarp, [('out_matrix_file', 'premat')])])
# coreg.connect([(fnirt2mni, applywarp, [('fieldcoeff_file', 'field_file')])])

# Connect outputs from other workflows here
preproc.connect([(selectfiles, coreg, [('anat', 'bet_anat.in_file')])])
preproc.connect([(fproc, coreg, [('moco.out_file', 'mean.in_file')])])


############################################## ######################### #################################################
#  /$$$$$$                                /$$      
# /$$__  $$                              | $$      
#| $$  \__/  /$$$$$$$  /$$$$$$  /$$   /$$| $$$$$$$ 
#|  $$$$$$  /$$_____/ /$$__  $$| $$  | $$| $$__  $$
# \____  $$| $$      | $$  \__/| $$  | $$| $$  \ $$
# /$$  \ $$| $$      | $$      | $$  | $$| $$  | $$
#|  $$$$$$/|  $$$$$$$| $$      |  $$$$$$/| $$$$$$$/
# \______/  \_______/|__/       \______/ |_______/ 
############################################## ######################### #################################################
# Motion scrubbing for the censoring of high motion time points

scrub = Workflow(name='scrub')

# Generate the Scrubbing Regressor
scrub_metrics = Node(MotionOutliers(dummy = 4,
                            out_file = 'FD_outliers.1D',
                            metric='fd',
                            threshold=0.4), name="scrub_metrics")

#regress out timepoints
scrub_frames = Node(Bandpass(highpass=0,
                         lowpass=99999,
                         outputtype='NIFTI_GZ'), name='scrub_frames')
scrub.connect([(scrub_metrics, scrub_frames, [('out_file', 'orthogonalize_file')])])
preproc.connect([(coreg, scrub, [('warp.output_image', 'scrub_frames.in_file')])])
preproc.connect([(selectfiles, scrub, [('func', 'scrub_metrics.in_file')])])


# mean image for remeaning after bandpass
premean = Node(TStat(args='-mean',
                   outputtype='NIFTI_GZ'), name='premean')
# remean the image
remean2 = Node(Calc(expr='a+b',
                    outputtype='NIFTI_GZ'), name='remean2')
scrub.connect([(scrub_frames, remean2, [('out_file', 'in_file_a')])])
scrub.connect([(premean, remean2, [('out_file', 'in_file_b')])])
preproc.connect([(coreg, scrub, [('warp.output_image', 'premean.in_file')])])

############################################## ######################### #################################################

#  ____  ____   ___  ____  ____  ____  ____   __   ____  ____
# (  _ \(  __) / __)(  _ \(  __)/ ___)/ ___) /  \ (  _ \/ ___)
#  )   / ) _) ( (_ \ )   / ) _) \___ \\___ \(  O ) )   /\___ \
# (__\_)(____) \___/(__\_)(____)(____/(____/ \__/ (__\_)(____/

############################################## ######################### #################################################
# Creates nuisance regressors like local white matter and cerebrospinal fluid signal
regressors = Workflow(name='regressors')

# Using registered structural image to create the masks for both WM and CSF
regbet = Node(BET(robust=True,
                  frac=0.37,
                  output_type='NIFTI_GZ'), name='regbet')

regseg = Node(FAST(img_type=1,
                   output_type='NIFTI_GZ',
                   no_pve=True,
                   no_bias=True,
                   segments=True), name='regseg')
regressors.connect([(regbet, regseg, [('out_file', 'in_files')])])
preproc.connect([(coreg, regressors, [('reg.warped_image', 'regbet.in_file')])])

############# CSF Regressor

# subtract subcortical GM from the CSF mask
subcortgm = Node(BinaryMaths(operation='sub',
                             operand_file='/Users/James/Psilodep/subcortical_gm_mask/subcortical_gm_mask_bin.nii.gz',
                             output_type='NIFTI_GZ',
                             args='-bin'), name='subcortgm')
regressors.connect([(regseg, subcortgm, [(('tissue_class_files', pickindex, 0), 'in_file')])])

# Fill the mask holes

fillcsf = Node(MaskTool(fill_holes=True,
                        outputtype='NIFTI_GZ'), name='fillcsf')
regressors.connect([(subcortgm, fillcsf, [('out_file', 'in_file')])])

# Erode the mask

erocsf = Node(MaskTool(outputtype='NIFTI_GZ',
                       dilate_inputs='-1'), name='erocsf')
regressors.connect([(fillcsf, erocsf, [('out_file', 'in_file')])])

# Take mean csf signal from functional image
meancsf = Node(ImageMeants(output_type='NIFTI_GZ'), name='meancsf')
regressors.connect([(erocsf, meancsf, [('out_file', 'mask')])])
preproc.connect([(coreg, regressors, [('warp.output_image', 'meancsf.in_file')])])

bpf_dt_csf = Node(CSFbpfdt(), name='bpf_dt_csf')
regressors.connect([(meancsf, bpf_dt_csf, [('out_file', 'in_file')])])

########## Local White Matter

# subtract subcortical gm
subcortgm2 = Node(BinaryMaths(operation='sub',
                              operand_file='/Users/James/Psilodep/subcortical_gm_mask/subcortical_gm_mask_bin.nii.gz',
                              output_type='NIFTI_GZ',
                              args='-bin'), name='subcortgm2')
regressors.connect([(regseg, subcortgm2, [(('tissue_class_files', pickindex, 2), 'in_file')])])

# fill mask
fillwm = Node(MaskTool(fill_holes=True,
                       outputtype='NIFTI_GZ'), name='fillwm')
regressors.connect([(subcortgm2, fillwm, [('out_file', 'in_file')])])

# erod mask
erowm = Node(MaskTool(outputtype='NIFTI_GZ',
                      dilate_inputs='-1'), name='erowm')
regressors.connect([(fillwm, erowm, [('out_file', 'in_file')])])

# generate local wm
localwm = Node(Localstat(neighborhood=('SPHERE', 25),
                         stat='mean',
                         nonmask=True,
                         outputtype='NIFTI_GZ'), name='localwm')
regressors.connect([(erowm, localwm, [('out_file', 'mask_file')])])
preproc.connect([(coreg, regressors, [('warp.output_image', 'localwm.in_file')])])

# bpf local wm
localwm_bpf = Node(Fourier(highpass=0.01,
                           lowpass=0.08,
                           args='-retrend',
                           outputtype='NIFTI_GZ'), name='loacwm_bpf')
regressors.connect([(localwm, localwm_bpf, [('out_file', 'in_file')])])

# detrend local wm

localwm_bpf_dt = Node(Detrend(args='-polort 2',
                              outputtype='NIFTI_GZ'), name='localwm_bpf_dt')
regressors.connect([(localwm_bpf, localwm_bpf_dt, [('out_file', 'in_file')])])

############################################## ######################### #################################################

#  $$$$$$\  $$\       $$$$$$$$\  $$$$$$\  $$\   $$\
# $$  __$$\ $$ |      $$  _____|$$  __$$\ $$$\  $$ |
# $$ /  \__|$$ |      $$ |      $$ /  $$ |$$$$\ $$ |
# $$ |      $$ |      $$$$$\    $$$$$$$$ |$$ $$\$$ |
# $$ |      $$ |      $$  __|   $$  __$$ |$$ \$$$$ |
# $$ |  $$\ $$ |      $$ |      $$ |  $$ |$$ |\$$$ |
# \$$$$$$  |$$$$$$$$\ $$$$$$$$\ $$ |  $$ |$$ | \$$ |
#  \______/ \________|\________|\__|  \__|\__|  \__|

############################################## ######################### #################################################

# create a mask for blurring filtering, and detrending

clean = Workflow(name='clean')

mask = Node(BET(mask=True,
                functional=True), name='mask')

mean_mask = Node(MCFLIRT(mean_vol=True,
                         output_type='NIFTI_GZ'), name="mean_mask")

dilf = Node(DilateImage(operation='max',
                        output_type='NIFTI_GZ'), name='dilf')
clean.connect([(mask, dilf, [('mask_file', 'in_file')])])
preproc.connect([(scrub, clean, [('remean2.out_file', 'mask.in_file')])])

fill = Node(MaskTool(in_file='/usr/local/fsl/data/standard/MNI152_T1_2mm_brain_mask.nii.gz',
                     fill_holes=True,
                     outputtype='NIFTI_GZ'), name='fill')

axb = Node(Calc(expr='a*b',
                outputtype='NIFTI_GZ'), name='axb')
clean.connect([(dilf, axb, [('out_file', 'in_file_a')])])
clean.connect([(fill, axb, [('out_file', 'in_file_b')])])

bxc = Node(Calc(expr='ispositive(a)*b',
                outputtype='NIFTI_GZ'), name='bxc')
clean.connect([(mean_mask, bxc, [('mean_img', 'in_file_a')])])
clean.connect([(axb, bxc, [('out_file', 'in_file_b')])])
preproc.connect([(scrub, clean, [('remean2.out_file', 'mean_mask.in_file')])])

#### BLUR, FOURIER, DETREND

blurinmask = Node(BlurInMask(fwhm=6,
                             outputtype='NIFTI_GZ'), name='blurinmask')
clean.connect([(bxc, blurinmask, [('out_file', 'mask')])])
preproc.connect([(scrub, clean, [('remean2.out_file', 'blurinmask.in_file')])])

fourier = Node(Fourier(highpass=0.01,
                       lowpass=0.08,
                       retrend=True,
                       outputtype='NIFTI_GZ'), name='fourier')
clean.connect([(blurinmask, fourier, [('out_file', 'in_file')])])

tstat = Node(TStat(args='-mean',
                   outputtype='NIFTI_GZ'), name='tstat')
clean.connect([(fourier, tstat, [('out_file', 'in_file')])])

detrend = Node(Detrend(args='-polort 2',
                       outputtype='NIFTI_GZ'), name='detrend')
clean.connect([(fourier, detrend, [('out_file', 'in_file')])])

remean = Node(Calc(expr='a+b',
                   outputtype='NIFTI_GZ'), name='remean')
clean.connect([(detrend, remean, [('out_file', 'in_file_a')])])
clean.connect([(tstat, remean, [('out_file', 'in_file_b')])])

concat = Node(ConcatModel(), name='concat')

# Removes nuisance regressors via regression function
clean_rs = Node(Bandpass(highpass=0,
                         lowpass=99999,
                         outputtype='NIFTI_GZ'), name='clean_rs')

clean.connect([(concat, clean_rs, [('out_file', 'orthogonalize_file')])])

remean1 = Node(Calc(expr='a+b',
                    outputtype='NIFTI_GZ'), name='remean1')
clean.connect([(clean_rs, remean1, [('out_file', 'in_file_a')])])
clean.connect([(tstat, remean1, [('out_file', 'in_file_b')])])

preproc.connect([(regressors, clean, [('bpf_dt_csf.out_file', 'concat.in_file_a')])])
preproc.connect([(fproc, clean, [('moco_bpfdt.out_file', 'concat.in_file_b')])])

preproc.connect([(regressors, clean, [('localwm_bpf_dt.out_file', 'clean_rs.orthogonalize_dset')])])
clean.connect([(remean, clean_rs, [('out_file', 'in_file')])])

##################################################### OUTPUTS ##########################################################

# connect what you want to the datasink

# preproc.connect([(coreg, datasink, [('func2struc.out_file', 'coreg.@func2struc')])])
# preproc.connect([(coreg, datasink, [('reg.warped_image', 'coreg.@struc2mni')])])
# preproc.connect([(coreg, datasink, [('warp.output_image', 'coreg.@func2mni')])])
# preproc.connect([(scrub, datasink, [('scrub_metrics.out_file', 'scrub.@scrub_times')])])
# preproc.connect([(scrub, datasink, [('scrub_metrics.out_metric_values', 'scrub.@FD_values')])])
# # preproc.connect([(regressors, datasink, [('scrub.out_metric_plot', 'scrub.@scrub_plot')])])
# preproc.connect([(fproc, datasink, [('moco_bpfdt.out_file', 'regressors.@Motion_regressor')])])
# preproc.connect([(regressors, datasink, [('bpf_dt_csf.out_file', 'regressors.@CSF_regressor')])])
# preproc.connect([(regressors, datasink, [('localwm_bpf_dt.out_file', 'regressors.@LocalWM_regressor')])])
# preproc.connect([(regressors, datasink, [('concat.out_file', 'regressors.@concat_model')])])
# preproc.connect([(regressors, datasink, [('clean_rs.out_file', 'regressors.@regressed_demeaned_rsfmri')])])
# preproc.connect([(clean, datasink, [('remean1.out_file', 'preprocessed.@preproc_rsfmri')])])

# Write graphs to visualize workflows

# fproc.write_graph(graph2use='flat', format='png', simple_form=True)
# fproc.write_graph(graph2use='colored', format='png', simple_form=True)

# coreg.write_graph(graph2use='flat', format='png', simple_form=True)
# coreg.write_graph(graph2use='colored', format='png', simple_form=True)

# scrub.write_graph(graph2use='flat', format='png', simple_form=True)
# scrub.write_graph(graph2use='colored', format='png', simple_form=True)

# regressors.write_graph(graph2use='flat', format='png', simple_form=True)
# regressors.write_graph(graph2use='colored', format='png', simple_form=True)

preproc.write_graph(graph2use='flat', format='png', simple_form=True)
preproc.write_graph(graph2use='colored', format='png', simple_form=True)

# preproc.run('MultiProc', plugin_args={'n_procs' : 6})


200511-15:44:26,39 nipype.workflow INFO:
	 Generated workflow graph: /Users/James/Desktop/preproc/preproc/graph.png (graph2use=flat, simple_form=True).
200511-15:44:26,192 nipype.workflow INFO:
	 Generated workflow graph: /Users/James/Desktop/preproc/preproc/graph.png (graph2use=colored, simple_form=True).


'/Users/James/Desktop/preproc/preproc/graph.png'

In [3]:
# preproc.run('MultiProc', plugin_args={'n_procs' : 4})

200511-15:44:27,109 nipype.workflow INFO:
	 Workflow preproc settings: ['check', 'execution', 'logging', 'monitoring']
200511-15:44:27,149 nipype.workflow INFO:
	 Running in parallel.
200511-15:44:27,154 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 2 jobs ready. Free memory (GB): 14.40/14.40, Free processors: 6/6.
200511-15:44:27,232 nipype.workflow INFO:
	 [Job 35] Cached (preproc.clean.fill).
200511-15:44:27,232 nipype.workflow INFO:
	 [Node] Setting-up "preproc.selectfiles" in "/Users/James/Desktop/preproc/preproc/_ses_1_subject_id_42/selectfiles".
200511-15:44:27,245 nipype.workflow INFO:
	 [Node] Running "selectfiles" ("nipype.interfaces.io.SelectFiles")
200511-15:44:27,253 nipype.workflow INFO:
	 [Node] Finished "preproc.selectfiles".
200511-15:44:29,154 nipype.workflow INFO:
	 [Job 0] Completed (preproc.selectfiles).
200511-15:44:29,157 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 3 jobs ready. Free memory (GB): 14.40/14.40, Free processors: 6/6.
200511-1

Process ForkProcess-6:
Process ForkProcess-1:
Process ForkProcess-2:
Process ForkProcess-5:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkProcess-4:
Traceback (most recent call last):
  File "/Users/James/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/James/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/James/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/James/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/James/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/James/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwarg

Traceback (most recent call last):
  File "/Users/James/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-5802bbc08d0b>", line 1, in <module>
    preproc.run('MultiProc', plugin_args={'n_procs' : 6})
  File "/Users/James/opt/anaconda3/lib/python3.7/site-packages/nipype/pipeline/engine/workflows.py", line 632, in run
    runner.run(execgraph, updatehash=updatehash, config=self.config)
  File "/Users/James/opt/anaconda3/lib/python3.7/site-packages/nipype/pipeline/plugins/base.py", line 189, in run
    sleep(max(0, sleep_til - time()))
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/James/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has

KeyboardInterrupt: 